House Characteristics
Default values from Gridlabd

In [9]:
import math
# Floor Area sq ft
A = 2500
# Floor Aspect Ratio
R = 1.5
# No of Stories
n = 1
# Ceiling Height ft
h = 8
# Exterior ceiling, fraction of total (ECR)
ECR = 1
# Exterior floor, fraction of total (EFR)	
EFR = 1
# Exterior wall, fraction of total (EWR)
EWR = 1
# Window/exterior wall area ratio (WWR)
WWR = 0.07
# Doors (nd)
nd = 4
# Area of 1 door (A1d) sq ft
A1d = 19.5
# Glazing Layers Integer 1,2,3
GL = 2
# Glazing Material (GM; glass, low-e glass)
GM = "low-e glass"
# Window frame (WF; none, aluminium, thermal break,wood,insulated)
WF = "wood"
# Glazing treatment (GT; clear, abs, refl, low-s, high-s)
GT = "clear"
# Window, exterior transmission coefficient (WET)
WET = 0.6
# R-value, walls (Rw) °F.ft².hr/Btu
Rw = 19
# R-Value, ceilings (Rc) °F.ft².hr/Btu
Rc = 30
# R-Value, floor (Rc) °F.ft².hr/Btu
Rf = 22
# R-Value, doors (Rc) °F.ft².hr/Btu
Rd = 5
# Infiltration volumetric air exchange rate (I)	L/hr
I = 0.5
# Interior/exterior wall surface ratio (IWR)
IWR = 1.5
# Interior surface heat transfer coefficient (hs) Btu/hr.°F.ft²
hs = 1.46
# Total thermal mass, per unit floor area (mf) Btu/°F.ft²
mf = 2.0
# Interior surface thermal mass, subtotal Btu/°F.ft²
mis = 1.5
# Solar gain fraction to mass (fs)
fs = 0.5
# Internal gain fraction to mass (fi)
fi = 0.5
# HVAC delivered fraction to mass (fm)
fm = 0
# Include solar quadrant (none, H(horizontal), N(north), S(south), E(east), W(west))
Solar_Q = "none"
# Windows U value Btu/hr-oF-ft2
U_g = 0.47 
# Nominal Solar Heat Gain Coefficient (SHGC) 
SHGC_window_nom = 0.67
# Gross exterior wall area
A_wt = 2*n*h*(1+R)*math.sqrt((A/n*R))
# Gross window Area
A_g = WWR*A_wt*EWR
# Total Door Area
A_d = nd*A1d
# The net exterior wall area
A_w = (A_wt - (A_g + A_d))*EWR
# The net exterior ceiling area 
A_c = (A/n)*ECR
# The net exterior floor area 
A_f = (A/n)*EFR
# The total heat loss coefficient (conductance) Btu/°F.hr
U_A = A_g*U_g + A_d/Rd + A_w/Rw + A_c/Rc + A_f/Rf + 0.018*A*h*I
# Solar Heat Gain Coefficient (SHGC) 
SHGC_window = SHGC_window_nom*WET*A_g
# Interior Mass Surface Conductance
H_m = hs*(A_w/EWR) + A_wt*IWR + (A_c*n)/ECR
# Total Air Mass
C_a = 3*(0.018*A*h)
# Total Thermal Mass
C_m = A*mf - 2*(0.018*A*h)

# Total internal Heat Gain as function of FloorArea Btu/hr
Q_internal = 167.09*(A)**0.442

# Solar radiation Btu/hr.ft2. Typical clear day incident radiation for 35° latitiude 
# and equal window areas in each of the eight cardinal directions 
Q_solar = 195

## ETP Heat balance Equation
### In Time Domain
Thermal conservation of Energy at Node $T_A$
$$Q_A  -  U_A (T_A-T_O)  -  H_M (T_A-T_M)  -  C_A \frac{dT_A}{dt} = 0 \qquad\qquad(1)$$
Thermal conservation of Energy at Node $T_M$
$$Q_M  -  H_M (T_M-T_A)  -  C_M \frac{dT_M}{dt} = 0\qquad\qquad(2)$$

State Space equation of States $T_M$ and $T_A$

$$ \begin{bmatrix} \dot{T_A} \\ \dot{T_M}  \end{bmatrix} = \begin{bmatrix} -(U_A+H_M)/C_A & H_M/C_A \\ H_M/C_M & -H_M/C_M \end{bmatrix} \begin{bmatrix} T_A \\ T_M \end{bmatrix} + \begin{bmatrix} 1/C_A \\ 0 \end{bmatrix}Q_A + \begin{bmatrix} U_A/C_A & 0 \\ 0 & 1/C_M \end{bmatrix} \begin{bmatrix} T_O \\ Q_M \end{bmatrix} $$

Considering $Q_M$ (addition from solar gain and internal gain to the house mass) and $T_O$ (Outdoor Temperature) as time variying noise to the system. $Q_A$ is the input from the HVAC system and internal gain to the indoor air mass.

### States of HVAC
1. HVAC Off
2. Heat on $Q_{heat} \in \{0,1\}$
3. Cool on $Q_{cool} \in \{0,1\}$
4. Auxilary Heat on $Q_{aux} \in \{0,1\}$

### Heat inputs from HVAC for various modes
1. HVAC off
    $$Q_H = 0$$
2. Heat on
    $$ $$

### In s Domain
Equation (1)
$$ T_A(s) = \frac{1}{s + (U_A+H_M)/C_A}\left(C_A U_A T_O(s) + C_A H_M T_M(s) + C_A Q_A(s) + C_A^2 T_{A0}\right)$$

Equation (2)

$$T_M(s) =  \frac{1}{s + H_M/C_M}\left(C_M H_M T_A(s) + C_M Q_M(s) + C_M^2 T_{M0}\right)$$

Combining Both

$$ T_A(s) = \frac{s+a_1}{s^2 + b_1s + b_2}\left(A_1T_O(s) + A_2Q_A(s) + A_3T_{A0}+ \frac{A_4Q_M(s)}{s+a_1}  + \frac{A_5T_{M0}}{s+a_1} \right)$$

where,

$a_1 = H_M/C_M \\$
$b_1 = (C_MU_A+C_MH_M+C_AH_M)/(C_AC_M)\\$
$b_2 = (U_AH_M+H_M^2 - (C_AC_MH_M)^2)/(C_AC_M)\\$
$A_1 = C_AU_A\\$
$A_2 = C_A\\$
$A_3 = C_A^2\\$
$A_4 = C_AC_MH_M\\$
$A_5 = C_M^2C_AH_M\\$

### In z domain via Bilinear Transform
$$ T_A\left(\frac{2(1-z^{-1})}{T(1+z^{-1})}\right) = $$


## Power consumed by HVAC system
P_hvac = F_electric * Heat_on * (Heating_Capacity/3.413)/Heating_COP + (Cooling_Capacity/3.413) * (1 + Latent_Cooling_Fraction)/Cooling_COP + P_fan